# graphql

> GitHub GraphQL API support for ghapi

- skip_showdoc: true
- skip_exec: true

In [ ]:
#| default_exp graphql

In [ ]:
#| export
import os
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

## GhGql Client

The `GhGql` class provides a lazy-loaded GraphQL client. The schema is fetched on first use, not at import time.

In [ ]:
#| export
#| export
class GhGql:
    "GitHub GraphQL client with lazy-loaded schema"
    def __init__(self, token=None):
        self.token = token or os.getenv('GITHUB_TOKEN')
        self._client = None
    
    @property
    def client(self):
        "Lazy-load the GraphQL client and schema"
        if self._client is None:
            if not self.token: raise ValueError("GITHUB_TOKEN not set")
            transport = RequestsHTTPTransport(
                url='https://api.github.com/graphql',
                headers={'Authorization': f'bearer {self.token}'}
            )
            self._client = Client(transport=transport, fetch_schema_from_transport=True)
            self._client.execute(gql('{ __typename }'))  # Trigger schema fetch
        return self._client
    
    @property
    def schema(self): return self.client.schema
    
    def __call__(self, query, variables=None):
        "Execute a GraphQL query"
        return self.client.execute(gql(query), variable_values=variables)
    
    def list_queries(self):
        "List all available top-level GraphQL query types"
        return list(self.schema.query_type.fields.keys())
    
    def query_args(self, name):
        "Show arguments for a query type"
        return self.schema.query_type.fields[name].args
    
    def type_fields(self, name):
        "List fields on a GraphQL type (use PascalCase, e.g. 'Repository')"
        return list(self.schema.type_map[name].fields.keys())

## Module-level convenience functions

These use a default client instance, similar to how powertools works.

In [ ]:
#| export
#| export
_default_client = None

def _get_client():
    "Get or create the default client"
    global _default_client
    if _default_client is None: _default_client = GhGql()
    return _default_client

def list_queries():
    "List all available top-level GraphQL query types"
    return _get_client().list_queries()

def query_args(name:str):
    "Show arguments for a query type"
    return _get_client().query_args(name)

def type_fields(name:str):
    "List fields on a GraphQL type (use PascalCase, e.g. 'Repository')"
    return _get_client().type_fields(name)

def gh_query(query:str, variables:dict[str,any]=None):
    "Execute a GraphQL query"
    return _get_client()(query, variables)

## Examples

In [ ]:
#| eval: false
# List available query types
list_queries()[:10]

In [ ]:
#| eval: false
# See what args a query takes
query_args('repository')

In [ ]:
#| eval: false
# See fields on a type
type_fields('Repository')[:10]

In [ ]:
#| eval: false
# Execute a query
gh_query('''
query($owner: String!, $name: String!) {
  repository(owner: $owner, name: $name) {
    description
    stargazerCount
  }
}
''', {'owner': 'AnswerDotAI', 'name': 'ghapi'})